In [5]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/63.9 kB ? eta -:--:--
   -------------------------------------- - 61.4/63.9 kB 1.6 MB/s eta 0:00:01
   -------------------------------------- - 61.4/63.9 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 63.9/63.9 kB 487.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/163.8 kB ? eta -:--:--
   ---------- ----------------------------- 41.0/163.8 kB ? eta -:--:--
   ---------- ----------------------------- 41.0/163.8 kB ? eta -:--:--
   -------------- ------------------------ 61.4/163.8 kB 409.6 kB/s eta 0:00:01
   ----------------------------------- -- 153.6/163.8 kB 762.6 kB/s eta 0:00:01
   ----------------------------------- -- 153.6/163.8 kB 762.6 kB/s eta 0:00:01
   -------------------------------------- 163.8/163.8 kB 577.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   -------

In [6]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   -------- ------------------------------ 30.7/147.9 kB 262.6 kB/s eta 0:00:01
   -------- ------------------------------ 30.7/147.9 kB 262.6 kB/s eta 0:00:01
   ---------- ---------------------------- 41.0/147.9 kB 179.6 kB/s eta 0:00:01
   ---------------- ---------------------- 61.4/147.9 kB 252.2 kB/s eta 0:00:01
   ------------------------------- ------ 122.9/147.9 kB 400.9 kB/s eta 0:00:01
   -------------------------------------- 147.9/147.9 kB 419.8 kB/s eta 0:00:00


In [7]:
import csv
import os
import time
import requests
from bs4 import BeautifulSoup

In [11]:
url =' https://www.buyrentkenya.com/houses-for-sale'
# set up the webscrapping agent for chrome
agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
# headers dictionary to send with the GET request
HEADERS = {'User-Agent': agent, 'Accept-Language': 'en-US, en'}

In [12]:
# get the data
data = requests.get(url, headers=HEADERS)

In [16]:
# creating a soup object
soup = BeautifulSoup(data.text, 'html.parser')

In [18]:
# Finding houses data in the div
# _ because clas is a reserved keyword in Python
results = soup.find_all('div', class_='listing_card')

In [33]:
# lists to store data in
titles = []
prices = []
descriptions = []
locations = []
no_of_bedrooms = []
no_of_bathrooms = []


In [49]:
# find house details
houses = soup.find_all('div', class_='listing-card')

In [53]:
# loop through first page houses
for page in range(1,180):

    for house in houses:
        title = house.find('span', class_='relative top-[2px] hidden md:inline').text.strip()
        price = house.find('p', class_='text-xl font-bold leading-7 text-grey-900').text.strip()
        description = house.find('a',class_='block truncate text-grey-500 no-underline').text.strip()
        location = house.find('p', class_='ml-1 truncate text-sm font-normal capitalize text-grey-650').text.strip()
        no_of_bedroom = house.find_all('span', class_='whitespace-nowrap font-normal')[0].text.strip()
        no_of_bathroom = house.find_all('span', class_='whitespace-nowrap font-normal')[1].text.strip()

        titles.append(title)
        locations.append(location)
        descriptions.append(description)
        prices.append(price)
        no_of_bedrooms.append(no_of_bedroom)
        no_of_bathrooms.append(no_of_bathroom)
        

In [55]:
# check whether all the data was scraped successfully
len(titles) == len(locations)

True